In [13]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import AutoImageProcessor, ResNetForImageClassification,ResNetConfig
import torch
from abc import ABC,abstractmethod

In [14]:
#TODO check if layers are actually being trained
#TODO add more models
#TODO compare metrics
#TODO test out prediction loop
#TODO learn how to use tensorboard

###DATASET PREP###

In [15]:
projectPath = r"Osteoporosis Knee X-ray"
pathList = []
labelList = []
dirList = os.listdir(projectPath)[:3]
for idx, x in enumerate(dirList):
    for xx in os.listdir(f"{projectPath}/{x}"):
        pathList.append(f"{projectPath}/{x}/{xx}")
        labelList.append(idx)

In [16]:
from PIL import Image
imageSizes = {}
for x in pathList:
    img = Image.open(x).size
    try:
        imageSizes[str(img)] = imageSizes[str(img)] + 1
    except KeyError:
        imageSizes[str(img)] = 1
imageSizes #varied image sizes, have to resize to 1024,1024

{'(1024, 1024)': 63,
 '(2180, 2660)': 61,
 '(2660, 2180)': 10,
 '(2430, 1994)': 94,
 '(2386, 1994)': 1,
 '(1994, 2430)': 2,
 '(2430, 1910)': 2,
 '(2402, 1994)': 1,
 '(2378, 1994)': 2,
 '(2362, 1994)': 1,
 '(2430, 1958)': 1,
 '(2398, 1994)': 1}

# old code starts here

In [17]:
import torchvision
import torch

class OsteoTorchDataset(torch.utils.data.Dataset):
    def __init__(self, itemsPath:list, labels:list, transform=None):
        
        self.itemsPath = itemsPath
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.itemsPath)

    def __getitem__(self,idx)->tuple[Image.Image,int]:
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        image = Image.open(self.itemsPath[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return torchvision.transforms.functional.pil_to_tensor(image), self.labels[idx]
        
    

transform = torchvision.transforms.Compose([torchvision.transforms.Resize([244,244])
                                            # ,torchvision.transforms.Grayscale()
                                            ])
osteoDataset = OsteoTorchDataset(pathList,labelList,transform)
                                            

In [18]:
from torch.utils.data.dataloader import DataLoader

train,val = torch.utils.data.random_split(osteoDataset,[0.7,0.3])#MAY BUG

trainLoader = DataLoader(train, batch_size = 16,shuffle=False,num_workers=0)
valLoader = DataLoader(val, batch_size = 16,shuffle=False,num_workers=0)

# New code starts here instead

In [19]:
# from dataWrapper import DataWrapper
# BUG still dosent work, does not fix the Num_workers issue
# loader = DataWrapper(pathList,labelList, batch_size = 16,shuffle=False,num_workers=15)

In [20]:
# import models here
from models import AlexNet

ImportError: cannot import name 'AlexNet' from 'models' (c:\Users\assaw\Documents\c_stuff\Python\machine learning\ISAIConference\models.py)

In [ ]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from torch import mode
import torch.nn.functional as F
import itertools

class ExperimentModel(L.LightningModule):
    def __init__(self,pretrainedModel,pretrainedPreprocess,outputSize,resize = False) -> None:
        super().__init__()
        self.model = pretrainedModel
        # if pretrainedPreprocess:
        #     self.preprocess = pretrainedPreprocess  
        # for param in self.model.parameters():
        #     param.requires_grad = False
        # self.resize = resize
        # if self.resize:
        #     self.linear1 = torch.nn.Linear(outputSize, 256)
        #     self.linear2 = torch.nn.Linear(256, 16)
        #     self.linear3 = torch.nn.Linear(16, 3) 
        self.valLog = []
        self.epoch = []
        self.cumLog = []
        
    def forward(self,input):
        out = self.model(input)
        # if self.resize:
        #     out = self.linear1(out)
        #     out = self.linear2(out)
        #     out = self.linear3(out)
        return out

        
    def training_step(self,batch):
        data,label = batch
        print(data.size())
        output = self(data.float())
        loss= F.cross_entropy(output,label)
        # self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data,label = batch
        output = self(data.float())
        loss= F.cross_entropy(output,label)
        self.cumLog.append(loss)
        # self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    # def predict(self)
    def on_test_epoch_end(self) -> None:
        # self.log("cust",self.linear3.weight.sum(),logger=True)
        return super().on_train_end()

    def get_self_variables(self):
        linear_variables = [x for x in dir(self) if 'linear' in x]
        return [getattr(self, x, None).parameters() for x in linear_variables]
        

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(),lr=0.001)
        # return torch.optim.AdamW(itertools.chain(self.model.parameters(),*self.get_self_variables()), lr=0.001)
        pass

    def predict_step(self, batch):
        data,label = batch
        return self(data.float())

# class ResNetCallbacks(L.Callback): #TODO implement callbacks
#     def on_validation_epoch_end(trainer, pl_module):
#         valLog.append(sum(self.cumLog)/len(self.cumlog))

    
# alexNet = ExperimentModel(AlexNet(num_classes=1000),None,1000,True)
alexNetNonMod = ExperimentModel(AlexNet(num_classes=3),None,3)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    verbose=True,        # Verbosity mode
    mode='min'           # Mode can be 'min', 'max', or 'auto'
)

logger = TensorBoardLogger("tb_logs", name="my_model")

trainer = L.Trainer(max_epochs = 20,accelerator='gpu', devices='auto', precision='16-mixed',callbacks=[early_stopping],logger=logger)
# trainer.fit(model=alexNet,train_dataloaders=loader.trainLoader,val_dataloaders=loader.valLoader)
trainer.fit(model=alexNetNonMod,train_dataloaders=trainLoader,val_dataloaders=valLoader)

Using 16bit Automatic Mixed Precision (AMP)
c:\Users\assaw\.conda\envs\AI\lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | AlexNet | 57.0 M | train
------------------------------------------
57.0 M    Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\assaw\.conda\envs\AI\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\assaw\.conda\envs\AI\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\assaw\.conda\envs\AI\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 11/11 [00:09<00:00,  1.13it/s, v_num=22, train_loss_step=1.040, val_loss_step=0.881, val_loss_epoch=1.030, train_loss_epoch=122.0]

...

Epoch 5: 100%|██████████| 11/11 [00:09<00:00,  1.13it/s, v_num=22, train_loss_step=1.050, val_loss_step=0.702, val_loss_epoch=0.865, train_loss_epoch=0.934]

Metric val_loss improved by 0.059 >= min_delta = 0.0. New best score: 0.865


Epoch 8: 100%|██████████| 11/11 [00:09<00:00,  1.16it/s, v_num=22, train_loss_step=1.110, val_loss_step=0.786, val_loss_epoch=0.922, train_loss_epoch=0.941]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.865. Signaling Trainer to stop.


Epoch 8: 100%|██████████| 11/11 [00:11<00:00,  0.92it/s, v_num=22, train_loss_step=1.110, val_loss_step=0.786, val_loss_epoch=0.922, train_loss_epoch=0.941]


In [ ]:
raise AssertionError("Stop here")
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    verbose=True,        # Verbosity mode
    mode='min'           # Mode can be 'min', 'max', or 'auto'
)

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = L.Trainer(max_epochs = 20,accelerator='gpu', devices='auto', precision='16-mixed',callbacks=[early_stopping],logger=logger)
trainer.fit(model=alexNetNonMod,train_dataloaders=trainLoader,val_dataloaders=valLoader)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs//

In [ ]:
data = []
label = []
valOutput = []
valLabel = []

from itertools import tee
testTrainer =  L.Trainer(accelerator='gpu', devices='auto')
out = testTrainer.predict(alexNet, valLoader)

loader1, loader2 = tee(valLoader)#BUG BUG BUG INCREDIBLY MEMORY INEFFICIENT DONOT RUN THIS ON ANYTHIGN BIGGER THAN 1000 IMAGES

for idx,x in enumerate(valLoader):
    data,label = x
    [valLabel.append(x) for x in label]

valOutput = trainer.predict(alexNet, loader2)

In [ ]:
label